In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110330

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,101540.8621
2018-02-28,78158.8760
2018-03-31,89307.1665
2018-04-30,49333.3365
2018-05-31,25224.8257
2018-06-30,49076.6872
2018-07-31,124792.2984
2018-08-31,108261.7461
2018-09-30,108749.8799


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    101540.8621
 2018-02-28     78158.8760
 2018-03-31     89307.1665
 2018-04-30     49333.3365
 2018-05-31     25224.8257
 2018-06-30     49076.6872
 2018-07-31    124792.2984
 2018-08-31    108261.7461
 2018-09-30    108749.8799
 2018-10-31    100742.0438
 2018-11-30     88507.5736
 2018-12-31    112676.3323
 2019-01-31    138739.1221
 2019-02-28    110824.7145
 2019-03-31    166086.8813
 2019-04-30     87482.7598
 2019-05-31     71057.3702
 2019-06-30     53980.1294
 2019-07-31     76124.4572
 2019-08-31    173528.8272
 2019-09-30     79423.0512
 2019-10-31     54546.8106
 2019-11-30     65253.7100
 2019-12-31    116178.7743
 2020-01-31    104524.2262
 2020-02-29     79373.9164
 2020-03-31    128743.1621
 2020-04-30     76254.5154
 2020-05-31     49786.8770
 2020-06-30     59335.0041
 2020-07-31    104600.9462
 2020-08-31    112160.9504
 2020-09-30     74761.1581
 2020-10-31     68827.5194
 2020-11-30     65953.0706
 2020-12-31    145253.4390

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -5.011508
p-value : 0.000021
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=994.978, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=994.314, Time=0.12 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=995.949, Time=0.07 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=997.574, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=998.376, Time=0.07 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.42 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=995.128, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=996.127, Time=0.04 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=997.454, Time=0.07 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=999.219, Time=0.10 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1000.330, Time=0.12 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=995.071, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=997.225, Time=0.10 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=998.788, Time=0.12 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=996.918, Time=0.38 sec
 ARIMA(3,0,0)(0,0,0)[1] inte

ARIMA(order=(0, 0, 1), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(0,0,1))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 1)   Log Likelihood                -494.120
Method:                       css-mle   S.D. of innovations          31094.037
Date:                Sat, 11 Sep 2021   AIC                            994.239
Time:                        03:58:43   BIC                            999.453
Sample:                             0   HQIC                           996.150
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.935e+04   6156.627     14.513      0.000    7.73e+04    1.01e+05
ma.L1.y        0.2900      0.163      1.784      0.074      -0.029       0.609
                                    Roots           

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([84212.46801045, 89352.46482532]), array([31094.03705503, 32375.41811389]), array([[ 23269.27524863, 145155.66077227],
       [ 25897.81133766, 152807.11831298]]))
